# Support Ticket Analysis

## Introduction

In this Jupyter Notebook, I will count the total number of support tickets available, as well as the number of tickets that reflect actual user queries (rather than onboarding tickets). 

# Counting the tickets

In [103]:
from collections import Counter
from atlassian import Jira
from datetime import datetime, timedelta
import os

import sys
sys.path.append("/Users/adam/Documents/University/Third Year Project/rag_project/src")

from polaris_rag.retrieval.document_loader import load_support_tickets
from polaris_rag.retrieval.document_preprocessor import build_jira_ticket_text

USERNAME = 'ac2650@cam.ac.uk'
PASSWORD = os.getenv('JIRA_API_TOKEN')


jira = Jira(
    url='https://ucam-rcs.atlassian.net',
    username=USERNAME,
    password=PASSWORD,
    cloud=True,
    api_version='3',
)


I first define some helper functions.

In [ ]:
def create_jira_jql_query(
        start_date: str, 
        end_date: str, 
        keywords: list[str] | None = None,
        project_key: str = "HPCSSUP",
    ) -> str:
    """Build a JQL string for resolved issues within a created-date window.

    The generated query always filters by the given project and `status = Resolved`,
    applies inclusive lower/upper bounds on the `created` field when `start_date`
    and/or `end_date` are provided, and orders results by most recent creation time.

    Parameters
    ----------
    start_date : str
        Inclusive lower bound for the `created` field in ISO date format
        ``YYYY-MM-DD``. If empty or ``None``, no lower bound is applied.
    end_date : str
        Inclusive upper bound for the `created` field in ISO date format
        ``YYYY-MM-DD``. If empty or ``None``, no upper bound is applied.
    project_key : str, optional
        Jira project key to filter on. Defaults to ``"HPCSSUP"``.

    Returns
    -------
    str
        A JQL string of the form:
        ``project = '<project_key>' AND status = Resolved [AND created >= 'YYYY-MM-DD'] [AND created <= 'YYYY-MM-DD'] ORDER BY created DESC``.

    Raises
    ------
    ValueError
        If ``start_date`` or ``end_date`` are provided but not in the
        ``YYYY-MM-DD`` format.

    Examples
    --------
    >>> create_jira_jql_query("2025-01-01", "2025-01-31", project_key="SUP")
    "project = 'SUP' AND status = Resolved AND created >= '2025-01-01' AND created <= '2025-01-31' ORDER BY created DESC"
    """
    
    jql_query = (
        f"project = '{project_key}' "
        f"AND status = Resolved "
    )
    if start_date:
        try:
            parsed_date = None

            try:
                parsed_date = datetime.strptime(start_date, "%Y-%m-%d")
            except ValueError:
                pass

            if parsed_date is None:
                raise ValueError(f"Invalid date format: {start_date}")
            
            formatted_date = parsed_date.strftime("%Y-%m-%d")
            jql_query += f"AND created >= '{formatted_date}' "
        except ValueError as e:
            raise ValueError(f"Invalid start_date provided: {start_date}") from e
        
    
    if end_date:
        try:
            parsed_date = None

            try:
                parsed_date = datetime.strptime(end_date, "%Y-%m-%d")
            except ValueError:
                pass

            if parsed_date is None:
                raise ValueError(f"Invalid date format: {end_date}")
            
            formatted_date = parsed_date.strftime("%Y-%m-%d")
            jql_query += f"AND created <= '{formatted_date}' "
        except ValueError as e:
            raise ValueError(f"Invalid end_date provided: {end_date}") from e
        
    keyword_clause = ""
        
    if keywords: 
        keyword_clause = " AND (" + " OR ".join([f'summary ~ "{keyword}"' for keyword in keywords]) + ")"
        jql_query += keyword_clause
        
    jql_query += f"ORDER BY created DESC"

    return jql_query

def count_support_tickets(
        username: str = None,
        password: str = None,
        start_date: str = None, 
        end_date: str = None, 
        project_key: str = "HPCSSUP",
        limit: int | None = 100,
    ) -> tuple[int, int]:
    """
    Count total and unique Jira support tickets within a date range.

    Parameters
    ----------
    start_date : str
        Inclusive lower bound for the issue creation date in ISO format
        (``YYYY-MM-DD``). If empty or ``None``, no lower bound is applied.
    end_date : str
        Inclusive upper bound for the issue creation date in ISO format
        (``YYYY-MM-DD``). If empty or ``None``, no upper bound is applied.
    project_key : str, optional
        Jira project key to query. Defaults to ``"HPCSSUP"``.
    limit : int or None, optional
        Maximum number of issues to retrieve. If ``None`` (default), all matching
        issues are returned.

    Returns
    -------
    list[dict]
        List of Jira issue dictionaries. Each issue contains standard Jira fields
        such as ``id``, ``key``, and ``fields``.

    Raises
    ------
    ValueError
        If ``start_date`` or ``end_date`` are provided but not valid ``YYYY-MM-DD`` strings.
    KeyError
        If Jira credentials are missing required keys (e.g., ``username`` or ``password``).
    """
    if not username or not password:
        raise KeyError("Jira credentials are not properly configured.")

    jira = Jira(
        url='https://ucam-rcs.atlassian.net',
        username=username,
        password=password,
        cloud=True,
        api_version='3',
    )

    jql_query = create_jira_jql_query(
        start_date=start_date,
        end_date=end_date,
        project_key=project_key,
    )

    next_page_token = None
    total_issue_count = 0
    unique_issue_count = 0
    summaries = []

    while True:
        if limit:
            if total_issue_count >= limit:
                break

        result = jira.enhanced_jql(
            jql_query,
            nextPageToken=next_page_token,
            expand=None,
        ) 

        temp_issues = result.get('issues', [])
        for issue in temp_issues:
            summaries.append(issue['fields']['summary'])
            total_issue_count += 1
        
        next_page_token = result.get('nextPageToken')

        if not next_page_token:
            break
    
    counts = Counter(summaries)
    unique_issues = {summary: count for summary, count in counts.items() if count == 1} 
    unique_issue_count += len(unique_issues)
    unique_summaries = list(unique_issues.keys())
    non_unique_summaries = [summary for summary in summaries if summary not in unique_summaries]

    if limit is not None:
        total_issue_count = min(total_issue_count, limit)

    return total_issue_count, unique_issue_count, unique_summaries, non_unique_summaries

In the cell below I load the most recent 100 support tickets and output their summaries. I also identify the summaries for which there exist multiple tickets. 

In [ ]:
counter = 0
summaries = []

start_year = 2026

for year in range(start_year, datetime.now().year + 1):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    total_count, unique_count, unique_summaries, non_unique_summaries = count_support_tickets(
        username=USERNAME,
        password=PASSWORD,
        start_date=start_date,
        end_date=end_date,
        limit=None,
    )
    
    print(f"Year: {year} - Total tickets: {total_count}, Unique tickets: {unique_count}")


In [ ]:
len(non_unique_summaries)

In [ ]:
set(non_unique_summaries)

Year: 2014 - Total tickets: 0, Unique tickets: 0
Year: 2015 - Total tickets: 1362, Unique tickets: 1140
Year: 2016 - Total tickets: 2709, Unique tickets: 2553
Year: 2017 - Total tickets: 3958, Unique tickets: 3801
Year: 2018 - Total tickets: 5475, Unique tickets: 5205
Year: 2019 - Total tickets: 6719, Unique tickets: 5536
Year: 2020 - Total tickets: 8596, Unique tickets: 7141
Year: 2021 - Total tickets: 9774, Unique tickets: 7802
Year: 2022 - Total tickets: 9765, Unique tickets: 7709

## Finding helpdesk tickets for the evaluation dataset

In [274]:
issues = load_support_tickets(
    username=USERNAME,
    password=PASSWORD,
    start_date="2025-01-01",
    limit=None,
    keywords=['quote']
)

In [268]:

filtered_issues = []

for issue in issues:
    summary = issue['fields']['summary']
    if 'has been added to project' in summary.lower():
        continue
    elif 'request for' in summary.lower():
        continue
    elif 'request user access' in summary.lower():
        continue
    elif 'request to terminate resource' in summary.lower() or 'has been removed from project' in summary.lower():
        continue
    filtered_issues.append(issue)

print(f"Filtered out {len(issues) - len(filtered_issues)} issues based on summary content.") 
issues = filtered_issues

Filtered out 0 issues based on summary content.


In [275]:
len(issues)

39

In [276]:
import html
import ipywidgets as widgets
from ipyevents import Event
from IPython.display import display

issue_texts = [build_jira_ticket_text(issue) for issue in issues]
state = {"idx": 0}

counter = widgets.HTML()
hint = widgets.HTML("<i>Use ← / → arrow keys (or buttons).</i>")

prev_btn = widgets.Button(description="← Previous")
next_btn = widgets.Button(description="Next →")

# Wrapped text area
text_view = widgets.HTML(
    layout=widgets.Layout(width="100%", height="453px")
)

def render():
    text = html.escape(issue_texts[state["idx"]])
    text_view.value = f"""
    <div style="
        width: 100%;
        height: 100%;
        overflow-y: auto;
        padding: 10px;
        border: 1px solid #ccc;
        white-space: pre-wrap;
        overflow-wrap: anywhere;
        word-break: break-word;
        box-sizing: border-box;
        font-family: monospace;
    ">{text}</div>
    """
    counter.value = f"<b>{state['idx'] + 1} / {len(issue_texts)}</b>"

def go_prev(_=None):
    if state["idx"] > 0:
        state["idx"] -= 1
        render()

def go_next(_=None):
    if state["idx"] < len(issue_texts) - 1:
        state["idx"] += 1
        render()

prev_btn.on_click(go_prev)
next_btn.on_click(go_next)

key_area = widgets.HTML(
    "<div style='padding:8px;border:1px solid #ccc;'>Click here, then use ← / → keys</div>"
)
key_event = Event(source=key_area, watched_events=["keydown"])

def handle_key(event):
    if event.get("key") == "ArrowLeft":
        go_prev()
    elif event.get("key") == "ArrowRight":
        go_next()

key_event.on_dom_event(handle_key)

ui = widgets.VBox([
    widgets.HBox([prev_btn, next_btn, counter]),
    hint,
    key_area,
    text_view
], layout=widgets.Layout(width="100%", height="500px"))

display(ui)
render()


## Counting tickets in 'eval_ticket_keys.txt'

In [282]:
with open('/Users/adam/Documents/University/Third Year Project/rag_project/data/test/eval_ticket_keys.txt', 'r') as f:
    eval_ticket_keys = [line.strip() for line in f.readlines()]

# find keys that have duplicates
counts = Counter(eval_ticket_keys)
duplicates = {key: count for key, count in counts.items() if count > 1} 

print(f"Duplicate ticket keys and their counts: {duplicates}")
print(f"Total unique ticket keys: {len(set(eval_ticket_keys))}")

Duplicate ticket keys and their counts: {}
Total unique ticket keys: 77


In [288]:
issues = load_support_tickets(
    username=USERNAME,
    password=PASSWORD,
    limit=None,
    keys=eval_ticket_keys,
)

print(len(issues))

77


In [289]:
for issue in issues:
    issue_text = build_jira_ticket_text(issue)
    with open("/Users/adam/Documents/University/Third Year Project/rag_project/data/test/evaluation_tickets.txt", "a") as f:
        f.write(issue_text + f"\n\n{"-" * 10}\n\n")

In [287]:
import html
import ipywidgets as widgets
from ipyevents import Event
from IPython.display import display

issue_texts = [build_jira_ticket_text(issue) for issue in issues]
state = {"idx": 0}

counter = widgets.HTML()
hint = widgets.HTML("<i>Use ← / → arrow keys (or buttons).</i>")

prev_btn = widgets.Button(description="← Previous")
next_btn = widgets.Button(description="Next →")

# Wrapped text area
text_view = widgets.HTML(
    layout=widgets.Layout(width="100%", height="453px")
)

def render():
    text = html.escape(issue_texts[state["idx"]])
    text_view.value = f"""
    <div style="
        width: 100%;
        height: 100%;
        overflow-y: auto;
        padding: 10px;
        border: 1px solid #ccc;
        white-space: pre-wrap;
        overflow-wrap: anywhere;
        word-break: break-word;
        box-sizing: border-box;
        font-family: monospace;
    ">{text}</div>
    """
    counter.value = f"<b>{state['idx'] + 1} / {len(issue_texts)}</b>"

def go_prev(_=None):
    if state["idx"] > 0:
        state["idx"] -= 1
        render()

def go_next(_=None):
    if state["idx"] < len(issue_texts) - 1:
        state["idx"] += 1
        render()

prev_btn.on_click(go_prev)
next_btn.on_click(go_next)

key_area = widgets.HTML(
    "<div style='padding:8px;border:1px solid #ccc;'>Click here, then use ← / → keys</div>"
)
key_event = Event(source=key_area, watched_events=["keydown"])

def handle_key(event):
    if event.get("key") == "ArrowLeft":
        go_prev()
    elif event.get("key") == "ArrowRight":
        go_next()

key_event.on_dom_event(handle_key)

ui = widgets.VBox([
    widgets.HBox([prev_btn, next_btn, counter]),
    hint,
    key_area,
    text_view
], layout=widgets.Layout(width="100%", height="500px"))

display(ui)
render()